# Project 5: Text Generation with Recurrent Neural Networks, LSTM, GRU, and Hyperas

In this notebook we're going to play with a new type of deep learning: Recurrent Neural Networks. The inteded output is a neural network capable of generating text in the style of whatever it was trained on. These networks obviously don't have thoughts of their own, so they don't form coherent thoughts, but they are eerily good at talking in the right style. Imagine an elementary school student making fun of their teacher saying, "Look at me, I'm Mr. S, math is important, stay in school, biology, mitochonria are the powerhouse of the cell, blah blah blah," and you'll have a good idea for the type of things these networks might produce.

These networks are specifically designed to deal with time series data. To understand the need for this, think about how you might design a traditional NN to process text.

First you'd need an input tensor of a particular shape. How would you determine the shape? How would you deal with different length sentences? What if I wanted to process a book?

Our second issue is how the NN would process the data. There's nothing in a NN architecture that looks at the order of the inputs. Let's say you built a NN that's predicting home prices. Sure, your NN has some idea of how square footage relates to school rating regarding home price, but it looks at both of these things at the same time. The order doesn't matter. But sometimes order does matter, and in those cases we need an RNN.

So we have a new type of NN that deals with time series data that can produce nonsense sentences. Who cares? 

The longer text generation that we're going to do is more fun than useful. But if you were to use it to just guess the next word, you can imagine where this might be used in auto-complete. You can also do interesting things with these networks like generate music.

## Ok, so what's an RNN?

An RNN is just a neural network that feeds it's output back into itself as an input. Imagine a for loop that just feeds the output of each iteration into the next cycle of the loop...that's really about it. Here's the common diagram you'll see for these netwroks:

![alt text](https://machinelearningblogcom.files.wordpress.com/2018/02/bildschirmfoto-2018-02-21-um-10-30-04.png?w=1400)

As the equal sign implies, those are just different ways of representing the same network. For the input at every time step (X), fed into the network (A), you get an output (h) that is both output and the input (X) for the next time step.

And just to reiterate: h is fed back into the network. I found the lack of label on the arrow feeding into the next timestep confusing whenever I was first reading about RNNs.

A Deep RNN is an RNN that has multiple NNs at ever time step. Honestly, you aren't going to see the term "Deep RNN" often. Most people just say RNN. But it is a common type that's used. It's going to look more like this:

![alt text](https://www.oreilly.com/library/view/neural-networks-and/9781492037354/assets/mlst_1412.png)

## What's going on in the network

Before literally addressing the technical details, let's talk about another problem that needs solving in our network: Memory. Think back to the Q-Learning notebook. Remember how our DQNN would forget how to play the early stage of a game after it got so good that it never faltered for thousands of iterations? Well, RNNs have a similar problem. If every time step is only determined by the last, then you quickly diverge from whatever was a few steps ealier.

Imagine your network starts with "The boy picked up the". What's the next word? Well, if your network doesn't remember anything past "the", then you're in trouble. You'll probably get a noun back, but there's slim chances that noun makes sense as something the boy would throw. You can see where this would quicly diverge into nonsense. 

We're going to solve this with a Long Short Term Memory (LSTM) cell. 

The actual meat of the processing is happening in the cell (the A in our diagram). There are a few cell types, but we're just going to focus on the common LSTM cell for now. Let's start with a scary picture and break it down:

![alt text](https://i.ytimg.com/vi/kMLl-TKaEnc/maxresdefault.jpg)

**Breaking the LSTM down**:

The first thing you'll want to note is that the LSTM outputs 2 states to the next cell (the two arrows). Each cell takes 2 inputs from the previous time step. 

We're going to call the top ouput arrow 'L' for 'long-term. This isn't what it's offically called, but it's not labeled in the diagram and it'll help us keep things straight as we walk through what's happening here. We're going to keep 'x' for the input for this timestep and 'h' as the output of the current timestep.

Here's the conceptual frameworks to get us going:

- The yellow boxes are neural network layers. They're labeled with their activation function. The O-shape is for sigmoid. Also notice that there is a tanh activation that's a pink circle.
    - Sigmoid acitvation functions output a value between 0 and 1. They center on 0.5 and approach 0 or 1 as the input values grow. 
    - Tanh activation functions are like sigmoid, but they range from -1 to 1 and center on 0.
    
![alt text](https://cdn-images-1.medium.com/max/800/1*f9erByySVjTjohfFdNkJYQ.jpeg)

- The pink circles with either a '+' or 'x' are either adding or multipling vectors. The multipliers are called the 'gates'.
    - Imagine what happens when an input comes into the bottom left sigmoid activation. Remember, sigmoids can output values from 0 to 1. Let's say this sigmoid zeros some values. The input coming into that multiplication gate in the top left will have values that are zero'd. That multiplier is acting as a gate for what values will be carried into the cell.
    
- If sigmoid activation are used to determine what's not important, tanh activations are deciding what to do with what is deemed important. Values that don't matter can still go to zero here, but the -1 to 1 range implies that we're using these to determine what we're doing with the information we're keeping.

- The last concpetual piece you'll want to focus on is that L (the top line) is meant for remembering past data. This is what solves the memory probelm from earlier.

**The Gates**

Our gates are actually named after their functions.

1. The top left gate is called the *forget gate*. Just like our example from earlier, the output (h) from the last cell and the input for this time step (x) are run through a sigmoid activation and the output of that activation is multiplied by the L from the previous LSTM cell. At this point, all of your values in L are either kept the same (multiplied by 1), forgotten (multiplied by zero), or tweaked (multiplied by something inbetween 0 and 1). 

2. The gate in the middle of the cell digram is the *input gate*. We run two activations before hitting this gate: the sigmoid and the tanh. Both activations are fed by both h and x. So, we process the the current inputs with the tanh, then decide what doesn't need to be remembered in the long term (this is the actual input gate - the sigmoid plus the multiplication), we then add what we want to remember to L. This is the L that's passed to the next LSTM.

3. The last gate, the *output gate*, is also fed by both a tanh and a sigmoid. We take the output from the last addition operation, another sigmoid activation of h and x, and multiply them to create our next h. In this step we're essentially combining out long term memory (L) with our current state (x and h sigmoid) to determine our next state. This is the h that's passed to the next LSTM and is the output for this time step.

Remember that the gates are just the multiplication operations. What we're really traning when we're training this network are the tanh and sigmoid activation function that feed the gates. The sigmoid for the forget gate needs totally different weights that the sigmoid that feeds the output gates.

## Working with text

### Tokenization 

Like any input into a neural network, we prefer our data to be one hot encoded or at least mapped to numbers in some way.

We can tokenize our data at a few levels. We have characters, words, and n-grams. (Assuming one hot method) Characters and words are just creating one hot encoding at that particualr level. If we just have the enligh alphabet for characters, your vecotr is going to have one 1 and 25 0s. If our corpus of text has 20000 words, a word based tokenization will have one 1 and 19999 0s. 

N-grams, or bag-of-words, is a little different. This isn't exactly going to give state of the art results, but it's common enough that you should know it. N-grams are overlapping chunks of sentences. If our entire dataset was just "The boy ran", we would break it down into "The", "The boy", "boy", "boy ran", and "ran". Obviously this has some drawbacks. It doesn't even preserve order.

### Embeddings

Let's say we tokenize our corpus at the word level. We can train a neural network layer that's whole purpose is to just help interpret the meaning of the words that we're feeding in. This is the embedding layer.

Embedding layers take the tokenzied vector that you feed in, let's say your 1/19999 vector, and instead turn it into a smaller vector with values between 0 and 1. So your 20000 size vector might come out looking something like `[0.4, 0.38, 0.86, 0.34]`.

The neat part about these vecotrs is that they can essentially represent meanings mathmatically. The classic example is that you can take the embedding vector for "king", subtract the embedding output for "man", and the output vector would land on a word like "royalty". You could then add the embedding for "woman", and the output will be the vector for "queen".

You can see where having an embedding layer in front of your RNN.

### You ready to code yet?

Time to get to the project part. As stated earlier, we'll be generating text. For our dataset I've decided to use President Trump's tweets.

I'd first like to say that I'm geniunely choosing Trump because I think he's both an objectively good and fun option. I don't mean this as political support or disapproval.

President Trump seems like an interesting subject for a few reasons. First, these types of RNNs aren't interesting when outputting a large amount of data. They tend to be better suited to smaller outputs. Trump has a massive corpus of short writings (tweets) to train on that are around the lenth we want for our outpus. Second, he has a distinctive style. I think it'll be obvious to anyone seeing the output of our network that we're trying to generate Trump-like speech (assuming this works...).

[I'll be pulling our database of tweets from here](http://www.trumptwitterarchive.com/archive)

In [1]:
import pandas as pd

tweet_data = pd.read_csv('trump_tweets.csv')
print(f'Colum Headings: {list(tweet_data.columns.values)}')
print(f'Shape: {tweet_data.shape}')

Colum Headings: ['source', 'text', 'created_at', 'retweet_count', 'favorite_count', 'is_retweet', 'id_str']
Shape: (23925, 7)


In [2]:
print(f'Oldest Tweet: {tweet_data.iloc[23924].created_at}: {tweet_data.iloc[23924].text}')
print('============================================================================')
print(f'Newest Tweet: {tweet_data.iloc[0].created_at}: {tweet_data.iloc[0].text}')

Oldest Tweet: 05-04-2009 18:54:25: Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!
Newest Tweet: 01-15-2019 16:11:31: Great being with the National Champion Clemson Tigers last night at the White House. Because of the Shutdown I served them massive amounts of Fast Food (I paid) over 1000 hamburgers etc. Within one hour it was all gone. Great guys and big eaters!


In [3]:
lengths = []
for index, tweet in tweet_data.iterrows():
    lengths.append(len(str(tweet['text']).split(' ')))

total_words = sum(lengths)
average_words = total_words / len(lengths)
print(f'Average Words Per Tweet: {average_words}')

Average Words Per Tweet: 19.802591431556948


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

entire_corpus = []
for index, tweet in tweet_data.iterrows():
    entire_corpus.append(str(tweet['text']))
    
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True,
                      split=' ',
                      char_level=False)

tokenizer.fit_on_texts(entire_corpus)

tokenized = tokenizer.texts_to_sequences(entire_corpus)

number_of_words = len(list(tokenizer.word_index)) + 1

print(f'Sample of word index: {list(tokenizer.word_index.items())[:5]}')
print(f'Number of word indexes: {number_of_words}')
print(f'Sample of word sequences: {tokenized[0]}')
print(f'Nubmer of word sequences: {len(tokenized)}')

Sample of word index: [('the', 1), ('to', 2), ('and', 3), ('a', 4), ('t', 5)]
Number of word indexes: 34056
Sample of word sequences: [18, 101, 21, 1, 162, 463, 4648, 5623, 103, 134, 23, 1, 308, 176, 148, 6, 1, 1234, 12, 2397, 102, 304, 1902, 6, 383, 917, 12, 586, 95, 2854, 12263, 852, 1614, 80, 1259, 22, 33, 35, 725, 18, 1949, 3, 59, 8921]
Nubmer of word sequences: 23925


In [5]:
import json
with open('trump_word_dict_tokenized.json', 'w') as file:
    output = json.dumps(tokenizer.word_index, indent=4)
    file.write(output)

In [6]:
reverse_index_of_word = tokenizer.index_word

reversing_tokens = ' '.join(reverse_index_of_word[word] for word in tokenized[100][:20])

print(reversing_tokens)

“former nytimes editor jill abramson rips paper's ‘unmistakably anti trump’ bias ”ms abramson is 100 correct horrible and totally dishonest


In [7]:
with open('trump_word_dict_reverse.json', 'w') as file:
    output = json.dumps(tokenizer.index_word, indent=4)
    file.write(output)

In [8]:
import numpy as np

features = []
labels = []

training_length = 3

# Iterate through the sequences of tokens
for sequence in tokenized:

    # Create multiple training examples from each sequence
    for index in range(training_length, len(sequence)):
        
        # Extract the features and label
        extract = sequence[index - training_length:index + 1]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])
    
features = np.array(features)

label_placeholder = np.zeros((len(features), number_of_words), dtype = np.int8)

for example_index, word_index in enumerate(labels):
    label_placeholder[example_index, word_index] = 1
    
labels = label_placeholder

print(features.shape)
print(labels.shape)

(435750, 3)
(435750, 34056)


In [9]:
print(f'Word at index 10000: {reverse_index_of_word[np.argmax(labels[10000])]}')

Word at index 10000: hunt


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = Sequential()

model.add(Embedding(input_dim=number_of_words,
              input_length = training_length,
              output_dim=16))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(32, activation='relu'))

model.add(Dense(number_of_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

x_train = features[:425000]
y_train = labels[:425000]
x_test = features[425000:]
y_test = labels[425000:]

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint('model1.h5', save_best_only=True, save_weights_only=False)
]

model.fit(x_train, y_train, 
            batch_size=4096, epochs=500,
            validation_data=(x_test, y_test),
            callbacks=callbacks)

C:\Users\seanm\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 425000 samples, validate on 10750 samples
Epoch 1/500
425000/425000 [==============================] - 68s 161us/step - loss: 8.8275 - acc: 0.0245 - val_loss: 7.5976 - val_acc: 0.0386
Epoch 2/500
425000/425000 [==============================] - 63s 149us/step - loss: 7.2088 - acc: 0.0372 - val_loss: 7.6006 - val_acc: 0.0386
Epoch 3/500
425000/425000 [==============================] - 62s 145us/step - loss: 7.1720 - acc: 0.0372 - val_loss: 7.6195 - val_acc: 0.0386
Epoch 4/500
425000/425000 [==============================] - 62s 147us/step - loss: 7.1539 - acc: 0.0372 - val_loss: 7.6264 - val_acc: 0.0386
Epoch 5/500
425000/425000 [==============================] - 62s 145us/step - loss: 7.1335 - acc: 0.0372 - val_loss: 7.6216 - val_acc: 0.0386
Epoch 6/500
425000/425000 [==============================] - 62s 146us/step - loss: 7.0927 - acc: 0.0372 - val_loss: 7.5818 - val_acc: 0.0386
Epoch 7/500
425000/425000 [==============================] - 62s 145us/step - loss: 7.0334 - acc:

In [12]:
from tensorflow.keras.models import load_model
import json

with open('trump_word_dict_reverse.json', 'r') as file:
    reverse_lookup = json.loads(file.read())
    
with open('trump_word_dict_tokenized.json', 'r') as file:
    tokenized = json.loads(file.read())
    
model = load_model('model1.h5')

C:\Users\seanm\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
import numpy as np

input_words = np.zeros((1, training_length), dtype = np.int8)

output = model.predict(input_words)

print(reverse_lookup[str(np.argmax(output))])

in


In [14]:
import random

output_words = []

input_words = [[]]

for x in range(training_length):
    input_words[0].append(random.randint(0,34056))
    
input_words = np.asarray(input_words)

def reweight_word(prediction, temperature=.9):
    new_pred_dist = np.log(prediction) / temperature
    new_pred_dist = np.exp(new_pred_dist)
    new_pred_dist = new_pred_dist / np.sum(new_pred_dist)
    new_index = np.argmax(new_pred_dist)
    return new_index

for i in range(20):
    word_oh = model.predict(input_words)
    weighted_index = reweight_word(word_oh)
    word = reverse_lookup[str(weighted_index)]
    # line above without function = reverse_lookup[str(np.argmax(word_oh))]
    output_words.append(word)

    new_input_placeholder = [[]]
    for i in range(training_length):
        index = i + 1
        if i < 2:
            new_input_placeholder[0].append(input_words[0][index])
        else:
            new_input_placeholder[0].append(weighted_index)

    input_words = np.asarray(new_input_placeholder)
    
output_tweet = ' '.join(output_words)

print(output_tweet)

to announce rivers years in the u s is a great job in the u s is a great job


## Results

Not Great. Our tweet is "at at 17 00 p m at the u s is a great job of the u s is a". You can see where it's going. It smells of a tweet from president Trump. But it's clearly not great. 

I played around with the temperature a bit. The network likes outputting "is a great job" all the way up to 1. At 1 it outputs total gibberish, usually consisting of the partial URLs that were embedded in tweets. Speaking of "embedded"....

The biggest problem I have with the network as-is is that there is no variety. It's really stuck on the whole 'great job' thing. If it output a variety tweets, all of similar caliber to the one listed above, I'd be less dissappointed.

# Using Pre-trained embeddings

For our 2nd attempt we're going to use pre-trained embedding layers. I think this will help in a few ways:

1. We'll get embeddings that have a much, much larger vocabulary and a better understanding of the relation between all of the words.
2. We'll have an actual dictionary that doesn't include things like partial URLs. I think these very sparsly represented, meaningless URLs and emoji encodings threw things off. 
3. I think our embedding layer may not have had enough data in both volume and variety to learn on. We don't have the problem with pre-trained embeddings.

We're going to use GloVe: Global Vectors for Word Representation embeddings, which was devleoped by a group out of Stanford in 2014. We're going to use a small GloVe pre-trained embedding that has 400K words.

I'm downloading the 'glove.6B.zip' embedding from [here](https://nlp.stanford.edu/projects/glove/). This package come with multiple output embedding sizes: 50, 100, 200, and 300. We'll be using 100.

This embedding with simply pass all zeros if it doesn't know a word.

We're going to have to reload and re-work our data.

from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd

tweet_data = pd.read_csv('trump_tweets.csv')

entire_corpus = []
for index, tweet in tweet_data.iterrows():
    entire_corpus.append(str(tweet['text']))
    
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True,
                      split=' ',
                      char_level=False)

tokenizer.fit_on_texts(entire_corpus)

word_index = tokenizer.word_index
reverse_index_word = tokenizer.index_word
number_of_words = len(word_index) + 1
word_counts = tokenizer.word_counts

tokenized = tokenizer.texts_to_sequences(entire_corpus)

features = []
labels = []

training_length = 3

for sequence in tokenized:
    for index in range(training_length, len(sequence)):
        extract = sequence[index - training_length:index + 1]
        features.append(extract[:-1])
        labels.append(extract[-1])
    
features = np.array(features)

label_placeholder = np.zeros((len(features), number_of_words), dtype = np.int8)

for example_index, word_index in enumerate(labels):
    label_placeholder[example_index, word_index] = 1
    
labels = label_placeholder

In [16]:
glove_vectors = 'glove.6B/glove.6B.100d.txt'
glove = np.loadtxt(glove_vectors, dtype='str', comments=None)
print(glove.shape)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2776: character maps to <undefined>

In [ ]:
vectors = glove[:, 1:].astype('float')
words = glove[:, 0]

# Clear the large embedding object from memory
del glove

In [ ]:
word_lookup = {word: vector for word, vector in zip(words, vectors)}

embedding_matrix = np.zeros((num_words, vectors.shape[1]))

not_found = 0

for index, word in enumerate(word_idx.keys()):
    vector = word_lookup.get(word, None)
    if vector is not None:
        embedding_matrix[index + 1, :] = vector
    else:
        not_found += 1
        
print(f'Words not found in embeddings: {not_found}')

In [ ]:
# More memory cleaning
import gc
gc.enable()
del vectors
gc.collect()

In [ ]:
#Each word is represented by 100 numbers with a number of words that can't be found. 
#We can find the closest words to a given word in embedding space using the cosine distance.
#This requires first normalizing the vectors to have a magnitude of 1.

# Normalize and convert nan to 0
embedding_matrix = embedding_matrix / np.linalg.norm(embedding_matrix, axis=1).reshape((-1, 1))
embedding_matrix = np.nan_to_num(embedding_matrix)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Masking
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

x_train = features[:425000]
y_train = labels[:425000]
x_test = features[425000:]
y_test = labels[425000:]

model = Sequential()

#New
model.add(Embedding(input_dim=number_of_words,
                    input_length = training_length,
                    output_dim=100,
                    weights=[embedding_matrix],
                    trainable=False,
                    mask_zero=True
                   ))

# New
model.add(Masking(mask_value=0.0))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(32, activation='relu'))

model.add(Dense(number_of_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint('model2.h5', save_best_only=True, save_weights_only=False)
]

model.fit(x_train, y_train, 
            batch_size=4096, epochs=500,
            validation_data=(x_test, y_test),
            callbacks=callbacks)

In [ ]:
from tensorflow.keras.models import load_model
import json

with open('trump_word_dict_reverse.json', 'r') as file:
    reverse_lookup = json.loads(file.read())
    
with open('trump_word_dict_tokenized.json', 'r') as file:
    tokenized = json.loads(file.read())
    
model = load_model('model2.h5')

In [ ]:
import random

output_words = []

input_words = [[]]

for x in range(training_length):
    input_words[0].append(random.randint(0,34056))
    
input_words = np.asarray(input_words)

def reweight_word(prediction, temperature=.9):
    new_pred_dist = np.log(prediction) / temperature
    new_pred_dist = np.exp(new_pred_dist)
    new_pred_dist = new_pred_dist / np.sum(new_pred_dist)
    new_index = np.argmax(new_pred_dist)
    return new_index

for i in range(20):
    word_oh = model.predict(input_words)
    weighted_index = reweight_word(word_oh)
    word = reverse_lookup[str(weighted_index)]
    # line above without function = reverse_lookup[str(np.argmax(word_oh))]
    output_words.append(word)

    new_input_placeholder = [[]]
    for i in range(training_length):
        index = i + 1
        if i < 2:
            new_input_placeholder[0].append(input_words[0][index])
        else:
            new_input_placeholder[0].append(weighted_index)

    input_words = np.asarray(new_input_placeholder)
    
output_tweet = ' '.join(output_words)

print(output_tweet)

## Results


# Hyperas